## Objective

Now that I know that my submodels perform a bit better than the model on the entire dataset, I can take advantage of the submodels and find the coefficients that are most associated with response variable.

In [1]:
CATEGORY_GROUPS_IN_QUESTION = [['Pick up Dead Animal', 'Animal Generic Request'],
['Abandoned Vehicles', 'Abandoned Bicycle'],
['Rodent Activity',	'Bed Bugs', 'Mice Infestation - Residential'],
['Sidewalk Repair', 'Sidewalk Repair (Make Safe)'],
['Needle Pickup'],
['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - ],Excessive  Insufficient'],
['Request for Pothole Repair'],
['Graffiti Removal']]

## Looking at coefs for Animal requests

Does this have statistically significant coefs, assuming homoskedacity and a linear predictor-response relationship and Normalized residuals and imperfect collinearity?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
from tqdm import tqdm

from utilities import remove_one_feature, scale

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

import os, sys
sys.path.append(os.path.join(os.path.dirname('.'), "../preprocessing"))
from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type, adjusted_r2, transform_school, get_vifs

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [5]:
df_orig = transform_school(df_orig)
df_orig.shape

../preprocessing/helper_functions.py:58: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  df.school = df.school.str.extract(r'(\d\d?)').astype(int)


(516406, 40)

## Filterering by Animal requests

In [6]:
df_orig = df_orig[df_orig.TYPE.isin(CATEGORY_GROUPS_IN_QUESTION[0])]
df_orig.shape

(8485, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [7]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

../preprocessing/helper_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  group[pd.np.abs(group - group.median()) > stds * group.std()] = pd.np.nan
/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.where(-key, value, inplace=True)


(8402, 40)

I'm removing ~1.5% of my rows.

## Remove `TYPE` col

In [8]:
df_outliers_removed.drop('TYPE', axis=1, inplace=True)

## Choosing columns

In [107]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'SubmittedPhoto']
cols_census = [
     'bedroom']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [108]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]
df.shape

(8402, 6)

## Removing NAs for cols like `school_std_dev`

In [109]:
aa = df.isnull().any().reset_index()
nas = aa[aa[0] == True]['index']
print nas

Series([], Name: index, dtype: object)


In [110]:
# this is a bad temporary band-aid
df = df.dropna(subset=nas.tolist())
df.shape

(8402, 6)

## Dummify

In [111]:
cols_to_dummify = [i for i in df.dtypes[df.dtypes == object].index if i != 'TYPE']
cols_to_dummify

[]

In [112]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

In [95]:
df_dummified.shape

(8402, 7)

## Checking for multicollinearity

In [96]:
df_dummified.head(1).T

,905400
COMPLETION_HOURS_LOG_10,0.0124857
SubmittedPhoto,False
poverty_pop_below_poverty_level,0.262473
bedroom,2
queue_wk,12873
queue_wk_open,1
is_description,True


In [97]:
get_vifs(df_dummified.drop(['SubmittedPhoto', 'is_description'], axis=1), 'COMPLETION_HOURS_LOG_10')

queue_wk                           1.333850
queue_wk_open                      1.323614
bedroom                            1.007058
poverty_pop_below_poverty_level    1.003156
dtype: float64

## Running model

In [78]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import string
from StringIO import StringIO


In [79]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [80]:
df_dummified.columns = [col.translate(None, string.punctuation).replace(' ', '') if col != 'COMPLETION_HOURS_LOG_10' else col for col in df_dummified.columns]

In [81]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

## Use LassoCV to find col subsets

In [82]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [83]:
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [ ]:
params = {'lassocv__alphas': make_alphas(-4, 4)}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=1)
model.fit(X_train, y_train);

In [ ]:
pd.DataFrame(model.cv_results_).T.iloc[2:5]

In [ ]:
model.best_params_

In [ ]:
'{} cols go to zero out of {}'.format(
    len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]),
    len(X_train.columns)
)

In [26]:
cols_zero = []

In [ ]:
cols_zero = list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0])
cols_zero

## Use subsetted cols to run lin reg

In [113]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop(['COMPLETION_HOURS_LOG_10'] + cols_zero, axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [114]:
col_list = ' + '.join(df_dummified.drop(['COMPLETION_HOURS_LOG_10'] + cols_zero, axis=1).columns)

est = smf.ols(
    'COMPLETION_HOURS_LOG_10 ~ {}'.format(col_list), 
    pd.concat([X_train, y_train], axis=1)).fit()

In [115]:
est.summary().tables[0]

Dep. Variable:,COMPLETION_HOURS_LOG_10,R-squared:,0.247
Model:,OLS,Adj. R-squared:,0.246
Method:,Least Squares,F-statistic:,439.8
Date:,"Thu, 23 Feb 2017",Prob (F-statistic):,0.00
Time:,07:28:13,Log-Likelihood:,-6236.8
No. Observations:,6721,AIC:,1.249e+04
Df Residuals:,6715,BIC:,1.253e+04
Df Model:,5,,
Covariance Type:,nonrobust,,


### Getting adjusted $R^2$ on test set

In [116]:
y_pred = est.predict(X_test)

In [117]:
adjusted_r2(y_test, y_pred, num_features=X_test.shape[1])

0.26634170973978044

In [118]:
mean_squared_error(y_test, y_pred)**0.5

0.60754908129435004

## Interpreting model

Which features are most associated with completion time?

In [119]:
df_results = pd.read_csv(StringIO(est.summary().tables[1].as_csv()), index_col=0).reset_index()
df_results.columns = ['coef_name'] + [i.rstrip().lstrip() for i in df_results.columns][1:]
df_results['coef_abs'] = pd.np.abs(df_results.coef)
df_results.coef_name = df_results.coef_name.map(lambda x: x.strip())
df_results = df_results.sort_values('P>|t|')
df_results.shape

(6, 7)

In [120]:
df_results.head(15)

,coef_name,coef,std err,t,P>|t|,[95.0% Conf. Int.],coef_abs
0,Intercept,1.446000,0.032000,45.408,0.000,1.384 1.508,1.446000
2,is_description[T.True],-0.903000,0.022000,-40.400,0.000,-0.947 -0.859,0.903000
4,queue_wk,-0.000016,0.000003,-6.185,0.000,-2.11e-05 -1.09e-05,0.000016
5,queue_wk_open,-0.002200,0.000000,-5.467,0.000,-0.003 -0.001,0.002200
3,bedroom,-0.026600,0.009000,-3.000,0.003,-0.044 -0.009,0.026600
1,SubmittedPhoto[T.True],-0.046800,0.027000,-1.731,0.084,-0.100 0.006,0.046800


In [121]:
scores = []

for col in X_train.columns:
    if col != 'Intercept':
        score = remove_one_feature([col], df_dummified, cols_zero)
        scores.append((col, score))
        
sorted(scores, key=lambda x: x[1])[::-1]        

[('queue_wk_open', 15),
 ('queue_wk', 15),
 ('SubmittedPhoto', 15),
 ('bedroom', 12.838),
 ('is_description', 9.539)]